# The practice dataset used for this tutorial is no longer supported, for details about the Harmony API see [this tutorial](https://nasa-openscapes.github.io/2021-Cloud-Hackathon/tutorials/07_Harmony_Subsetting.html#data-subsetting-and-transformation-services-in-the-cloud) from the 2021 Cloud Hackathon or [this tutorial](https://github.com/nasa/harmony-py/blob/main/examples/intro_tutorial.ipynb) introducing the Harmony-py library.
## Harmony EOSS API Tutorial

## Before you start

Before you beginning this tutorial, make sure you have an account in the Earthdata Login, which is required to access data from the NASA Earthdata system. Please visit https://urs.earthdata.nasa.gov to register for an Earthdata Login account. It is free to create and only takes a moment to set up.

You will also need a netrc file containing your NASA Earthdata Login credentials in order to execute this notebook. A netrc file can be created manually within text editor and saved to your home directory. For additional information see: [Authentication for NASA Earthdata](https://nasa-openscapes.github.io/2021-Cloud-Hackathon/tutorials/04_NASA_Earthdata_Authentication.html#authentication-via-netrc-file).

## Build the EOSS Root URL

Next we will build a URL for the EOSS service for a given granule.  To get data using the service, you need 
a [CMR](https://cmr.earthdata.nasa.gov) collection ID for a supported collection and the ID of a
granule within that collection.

By convention, all Harmony services are accessed through `<harmony_root>/<collection_id>/<service_name>`

In [1]:
harmony_root = 'https://harmony.earthdata.nasa.gov'
config = {
    'collection_id': 'C1233800302-EEDTEST',
    'eoss_version': '0.1.0'
}
eoss_collection_root = harmony_root+'/{collection_id}/eoss/{eoss_version}/items/'.format(**config)
print(eoss_collection_root)

https://harmony.earthdata.nasa.gov/C1233800302-EEDTEST/eoss/0.1.0/items/


## Variable Subset of a Granule

We can now build onto the root URL in order to actually perform a transformation.  The first transformation is a variable subset of a selected granule.  _At this time, this requires discovering the granule id and variable id from CMR_.  That information can then be appended to the root URL and used to call Harmony with the help of the `request` library.

Harmony stages transformed data in S3 to make it easy to do additional processing in the cloud. The response that Harmony returns is actually a redirect to the S3 location where your data is staged.  Should you call Harmony in a tool that follows redirects, like your web browser, your file will be seamlessly downloaded locally for you.  However, should you desire to do additional processing in AWS, you have that option as well by simply looking at the redirected URL.  The code snippet below uses "geturl()" to show the URL of your staged data.

In [ ]:
varSubsetConfig = {
    'granule_id' : 'G2524192900-POCLOUD',
    'variable_id' : 'red_var'
}
eoss_var_subset_url = eoss_collection_root+'{granule_id}/?rangeSubset={variable_id}'.format(**varSubsetConfig)

print('Request URL', eoss_var_subset_url)

with request.urlopen(eoss_var_subset_url) as response:
    print('URL for data staged in S3:', response.geturl())

## Add on a spatial subset

The second transformation is a spatial subset of a selected granule. This can be combined with the request we already built above by simply specifying a bounding box.

In [ ]:
spatialSubsetConfig = {
    'west' : '-128',
    'south' : '23',
    'east' : '-63',
    'north' : '47'
}
eoss_spatial_subset_url = eoss_var_subset_url+'&bbox={west},{south},{east},{north}'.format(**spatialSubsetConfig)

print('Request URL', eoss_spatial_subset_url)

with request.urlopen(eoss_spatial_subset_url) as response:
    print('URL for data staged in S3:', response.geturl())

## Reprojection

The third transformation is a reprojection of the data. This can be combined with the requests we already built above by simply specifying a coordinate reference system.  Coordinate reference systems are identified by a common name, EPSG code, or URI. Today, this is based on reference systems supported by gdal. Examples include: 'CRS:84', 'EPSG:32611'.

In [ ]:
reprojectionConfig = {
    'crs' : 'EPSG:32611'
}
eoss_reprojection_url = eoss_spatial_subset_url+'&crs={crs}'.format(**reprojectionConfig)

print('Request URL', eoss_reprojection_url)

with request.urlopen(eoss_reprojection_url) as response:
    print('URL for data staged in S3:', response.geturl())


## Reformatting

Next is a reformatting of the output file of the data. This can be combined with the requests we already built above by simply specifying a format.  Examples include: image/tiff', 'image/png'

In [ ]:
reformattingConfig = {
    'format' : 'image/png'
}
eoss_reformatting_url = eoss_reprojection_url+'&format={format}'.format(**reformattingConfig)

print('Request URL', eoss_reformatting_url)

with request.urlopen(eoss_reformatting_url) as response:
    print('URL for data staged in S3:', response.geturl())


## Continue Exploring

Harmony's specification is available online.  Feel free to read more and continue exploring how to use Harmony. https://harmony.uat.earthdata.nasa.gov/docs/eoss/0.1.0/spec